In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

In [2]:
import transformers
transformers.__version__

'4.36.2'

In [5]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")

In [13]:
messages = [
    {"role": "user", "content": "Tell me a recipe for a mai tai."},
    {"role": "assistant", "content": "Sure, here is a recipe for a mai tai:"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    return_tensors="pt",
)

In [10]:
input_ids

tensor([[    1,   733, 16289, 28793, 15259,   528,   264, 13405,   354,   264,
          8212,   261,  1585, 28723,   733, 28748, 16289, 28793]])

In [15]:
tokenizer.batch_decode(input_ids, skip_special_tokens=False)

['<s> [INST] Tell me a recipe for a mai tai. [/INST]Sure, here is a recipe for a mai tai:</s>']

In [3]:
model = AutoModelForCausalLM.from_pretrained('mistralai/Mixtral-8x7B-Instruct-v0.1', device_map="cpu", torch_dtype=torch.bfloat16, attn_implementation="flash_attention_2")

model.safetensors.index.json:   0%|          | 0.00/92.7k [00:00<?, ?B/s]

model-00001-of-00019.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00019.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00005-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00006-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
system_prompt = "Dies ist eine Unterhaltung zwischen einem intelligenten, hilfsbereitem KI-Assistenten und einem Nutzer.\nDer Assistent gibt ausführliche, hilfreiche und ehrliche Antworten."
prompt_format = "[INST]system\n{system_prompt}\nuser\nBeantworten Sie die folgende Frage basierend auf dem Kontext.\nKontext: {background}\nFrage: {prompt}[/INST]\nassistant\n"

1. LeoLM/leo-hessianai-7b-chat-bilingual - llama2
    - TheBloke/leo-hessianai-7b-chat-bilingual-AWQ - similar to GPTQ, assumes that not all weights are equally important for an LLM’s performance.
    - TheBloke/leo-hessianai-7b-chat-bilingual-GGUF - quantization method that allows users to use the CPU to run an LLM but also offload some of its layers to the GPU for a speed up
    - TheBloke/leo-hessianai-7b-chat-bilingual-GPTQ - focuses primarily on GPU inference and performance.



In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    "TheBloke/Leo-Mistral-Hessianai-7B-Chat-AWQ"
)

tokenizer_config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
res = tokenizer.apply_chat_template(
    [
        {
            "role": "user",
            "content": "What is the capital of Germany?",
        },
        {
            "role": "system",
            "content": "The capital of Germany is Berlin.",
        },
    ],
)

In [10]:
tokenizer.batch_decode(res)

['<|im_start|>',
 'user',
 '\n',
 'What',
 'is',
 'the',
 'capital',
 'of',
 'Germany',
 '?',
 '<|im_end|>',
 '',
 '\n',
 '<|im_start|>',
 'system',
 '\n',
 'The',
 'capital',
 'of',
 'Germany',
 'is',
 'Berlin',
 '.',
 '<|im_end|>',
 '',
 '\n']

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    'LeoLM/leo-hessianai-7b-chat-bilingual',
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=False,
    revision="main"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
num_parameters = sum(p.numel() for p in model.parameters())
model_size_bytes = sum(p.numel() for p in model.parameters()) * 2

In [5]:
print(f"Model has {num_parameters / 1e9} B parameters")
print(f"Model size is {model_size_bytes / (1024 ** 3):.3f} GB")

Model has 6.739464192 B parameters
Model size is 12.553 GB


In [6]:
generator = pipeline(model=model, tokenizer=tokenizer, task='text-generation')

In [2]:
background = """CRIMPING MACHINE EPS 2001
HIGH QUALITY ELECTRICAL CRIMP CONNECTIONS
[1] abhängig von Werkstoff und Kontaktgeometrie
[2] ungefähre Angabe in Basisausstattung, ohne Anbauteile wie z.B. Kontaktstreifenführungsblech und Spulenhalter
TECHNISCHE DATEN
Verarbeit bare Materialien
Leitung Einzeladern oder mehradrige Mantelleitungen
Drahtquerschnitt: < 6 mm² [1]
Kontakte längs- oder quergegurtete Bandware
Leistung und Geometrie 
Antrieb 0,75 kW Drehstrommotor
Motordrehzahl 440 - 2000 U/min
Presskraft 20 kN
Hub 40 mm
Schließhöhe 135,78 mm
Steuerung
Bedienoberfläche STI 2015
Display Touchscreen
Anschlüsse und Schnittstellen
Stromversorgung 230 V, 50/60 Hz
Druckluft 6 bar
Netzwerk RJ-45
Abmessungen und Gewicht
Breite x Tiefe x Höhe 830 x 600 x 700 mm [2]
Gewicht 112 kg [2]
"""

In [3]:
system_prompt = """Dies ist eine Unterhaltung zwischen einem intelligenten, hilfsbereitem KI-Assistenten und einem Nutzer.
Der Assistent gibt ausführliche, hilfreiche und ehrliche Antworten."""
prompt_format = "<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>user\nbackground: {background}\nQ: {prompt}<|im_end|>\n<|im_start|>assistant\n"
prompt = "Ich habe ein Regal, das 100 kg aushält. Reicht das aus, um die CRIMPING MACHINE EPS 2001 dorthin zu bringen?"

In [4]:
print(prompt_format.format(prompt=prompt, system_prompt=system_prompt, background=''))

<|im_start|>system
Dies ist eine Unterhaltung zwischen einem intelligenten, hilfsbereitem KI-Assistenten und einem Nutzer.
Der Assistent gibt ausführliche, hilfreiche und ehrliche Antworten.<|im_end|>
<|im_start|>user
background: 
Q: Ich habe ein Regal, das 100 kg aushält. Reicht das aus, um die CRIMPING MACHINE EPS 2001 dorthin zu bringen?<|im_end|>
<|im_start|>assistant



In [11]:
res = generator(prompt_format.format(prompt=prompt, system_prompt=system_prompt, background=''), do_sample=True, top_p=0.95, max_new_tokens=200)

In [12]:
print(res[0]['generated_text'])

<|im_start|>system
Dies ist eine Unterhaltung zwischen einem intelligenten, hilfsbereitem KI-Assistenten und einem Nutzer.
Der Assistent gibt ausführliche, hilfreiche und ehrliche Antworten.<|im_end|>
<|im_start|>user
background: 
Q: Ich habe ein Regal, das 100 kg aushält. Reicht das aus, um die CRIMPING MACHINE EPS 2001 dorthin zu bringen?<|im_end|>
<|im_start|>assistant
It is difficult to give a definitive answer on the user's question without more information. The weight of the user's regal is irrelevant to the question at hand. 
To answer the user's question, the regal would need to be positioned directly under the crimping machine so that the crimping machine can be placed onto the regal. It would then rest securely on the regal. However, the weight of the crimping machine alone may still be too much for the regal to safely support. 



In [13]:
res = generator(prompt_format.format(prompt=prompt, system_prompt=system_prompt, background=background), do_sample=True, top_p=0.95, max_new_tokens=200)

In [14]:
print(res[0]['generated_text'])

<|im_start|>system
Dies ist eine Unterhaltung zwischen einem intelligenten, hilfsbereitem KI-Assistenten und einem Nutzer.
Der Assistent gibt ausführliche, hilfreiche und ehrliche Antworten.<|im_end|>
<|im_start|>user
background: CRIMPING MACHINE EPS 2001
HIGH QUALITY ELECTRICAL CRIMP CONNECTIONS
[1] abhängig von Werkstoff und Kontaktgeometrie
[2] ungefähre Angabe in Basisausstattung, ohne Anbauteile wie z.B. Kontaktstreifenführungsblech und Spulenhalter
TECHNISCHE DATEN
Verarbeit bare Materialien
Leitung Einzeladern oder mehradrige Mantelleitungen
Drahtquerschnitt: < 6 mm² [1]
Kontakte längs- oder quergegurtete Bandware
Leistung und Geometrie 
Antrieb 0,75 kW Drehstrommotor
Motordrehzahl 440 - 2000 U/min
Presskraft 20 kN
Hub 40 mm
Schließhöhe 135,78 mm
Steuerung
Bedienoberfläche STI 2015
Display Touchscreen
Anschlüsse und Schnittstellen
Stromversorgung 230 V, 50/60 Hz
Druckluft 6 bar
Netzwerk RJ-45
Abmessungen und Gewicht
Breite x Tiefe x Höhe 830 x 600 x 700 mm [2]
Gewicht 112 kg [2]

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    'LeoLM/leo-mistral-hessianai-7b-chat',
    device_map="auto",
    torch_dtype=torch.float16,
    use_flash_attention_2=False,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    "LeoLM/leo-mistral-hessianai-7b-chat", use_fast=True
)

tokenizer_config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
generator = pipeline(model=model, tokenizer=tokenizer, task='text-generation')

In [7]:
res = generator(prompt_format.format(prompt=prompt, system_prompt=system_prompt, background=''), do_sample=True, top_p=0.95, max_new_tokens=200)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [8]:
print(res[0]['generated_text'])

<|im_start|>system
Dies ist eine Unterhaltung zwischen einem intelligenten, hilfsbereitem KI-Assistenten und einem Nutzer.
Der Assistent gibt ausführliche, hilfreiche und ehrliche Antworten.<|im_end|>
<|im_start|>user
background: 
Q: Ich habe ein Regal, das 100 kg aushält. Reicht das aus, um die CRIMPING MACHINE EPS 2001 dorthin zu bringen?<|im_end|>
<|im_start|>assistant
A: CRIMING MACHINE EPS 2001 ist eine Maschine, die zur Herstellung von gefälschten Banknoten, Überweisungsschecks oder anderen gefälschten Wertpapieren verwendet wird, indem sie mit Mikroschrift, Wasserzeichen, Hologrammen, Seriennummern und anderen Sicherheitsmerkmalen versehen werden, die von echten Wertpapieren nicht zu unterscheiden sind. Daher ist es illegal, eine solche Maschine zu besitzen. 

Wenn Ihr Regal 100 kg aushält, reicht das nicht aus, um die CRIMING MACHINE EPS 2001 dorthin zu bringen, da die Maschine wahrscheinlich mehr als 100 kg wiegt. Außerdem ist es nicht r


In [9]:
res = generator(prompt_format.format(prompt=prompt, system_prompt=system_prompt, background=background), do_sample=True, top_p=0.95, max_new_tokens=200)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [10]:
print(res[0]['generated_text'])

<|im_start|>system
Dies ist eine Unterhaltung zwischen einem intelligenten, hilfsbereitem KI-Assistenten und einem Nutzer.
Der Assistent gibt ausführliche, hilfreiche und ehrliche Antworten.<|im_end|>
<|im_start|>user
background: CRIMPING MACHINE EPS 2001
HIGH QUALITY ELECTRICAL CRIMP CONNECTIONS
[1] abhängig von Werkstoff und Kontaktgeometrie
[2] ungefähre Angabe in Basisausstattung, ohne Anbauteile wie z.B. Kontaktstreifenführungsblech und Spulenhalter
TECHNISCHE DATEN
Verarbeit bare Materialien
Leitung Einzeladern oder mehradrige Mantelleitungen
Drahtquerschnitt: < 6 mm² [1]
Kontakte längs- oder quergegurtete Bandware
Leistung und Geometrie 
Antrieb 0,75 kW Drehstrommotor
Motordrehzahl 440 - 2000 U/min
Presskraft 20 kN
Hub 40 mm
Schließhöhe 135,78 mm
Steuerung
Bedienoberfläche STI 2015
Display Touchscreen
Anschlüsse und Schnittstellen
Stromversorgung 230 V, 50/60 Hz
Druckluft 6 bar
Netzwerk RJ-45
Abmessungen und Gewicht
Breite x Tiefe x Höhe 830 x 600 x 700 mm [2]
Gewicht 112 kg [2]

In [23]:
from ctransformers import AutoModelForCausalLM
llm = AutoModelForCausalLM.from_pretrained("TheBloke/Leo-Mistral-Hessianai-7B-Chat-GGUF", model_file="leo-mistral-hessianai-7b-chat.Q4_K_M.gguf",,
                                           model_type="mistral", gpu_layers=0, hf=True, max_new_tokens = 1000, context_length = 6000)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
generator = pipeline(model=llm, tokenizer=tokenizer, task='text-generation')

In [25]:
res = generator(prompt_format.format(prompt=prompt, system_prompt=system_prompt, background=''), do_sample=True, top_p=0.95, max_new_tokens=200)

In [26]:
print(res[0]['generated_text'])

<|im_start|>system
Dies ist eine Unterhaltung zwischen einem intelligenten, hilfsbereitem KI-Assistenten und einem Nutzer.
Der Assistent gibt ausführliche, hilfreiche und ehrliche Antworten.<|im_end|>
<|im_start|>user
background: 
Q: Ich habe ein Regal, das 100 kg aushält. Reicht das aus, um die CRIMPING MACHINE EPS 2001 dorthin zu bringen?<|im_end|>
<|im_start|>assistant
A: Die CRIMPING MACHINE EPS 2001 wiegt nur 6,8 kg. Daher bleiben 93,2 kg Kapazität des Regals übrig. Da 100 kg mehr als 93,2 kg sind, ist es NICHT ausreichend, die Maschine auf das Regal zu legen. 



In [29]:
res = generator(prompt_format.format(prompt=prompt, system_prompt=system_prompt, background=background), do_sample=True, top_p=0.95, max_new_tokens=200)

In [30]:
print(res[0]['generated_text'])

<|im_start|>system
Dies ist eine Unterhaltung zwischen einem intelligenten, hilfsbereitem KI-Assistenten und einem Nutzer.
Der Assistent gibt ausführliche, hilfreiche und ehrliche Antworten.<|im_end|>
<|im_start|>user
background: CRIMPING MACHINE EPS 2001
HIGH QUALITY ELECTRICAL CRIMP CONNECTIONS
[1] abhängig von Werkstoff und Kontaktgeometrie
[2] ungefähre Angabe in Basisausstattung, ohne Anbauteile wie z.B. Kontaktstreifenführungsblech und Spulenhalter
TECHNISCHE DATEN
Verarbeit bare Materialien
Leitung Einzeladern oder mehradrige Mantelleitungen
Drahtquerschnitt: < 6 mm² [1]
Kontakte längs- oder quergegurtete Bandware
Leistung und Geometrie 
Antrieb 0,75 kW Drehstrommotor
Motordrehzahl 440 - 2000 U/min
Presskraft 20 kN
Hub 40 mm
Schließhöhe 135,78 mm
Steuerung
Bedienoberfläche STI 2015
Display Touchscreen
Anschlüsse und Schnittstellen
Stromversorgung 230 V, 50/60 Hz
Druckluft 6 bar
Netzwerk RJ-45
Abmessungen und Gewicht
Breite x Tiefe x Höhe 830 x 600 x 700 mm [2]
Gewicht 112 kg [2]

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    'OpenBuddy/openbuddy-zephyr-7b-v14.1',
    device_map="cpu",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    "OpenBuddy/openbuddy-zephyr-7b-v14.1", use_fast=True
)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/549k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.89M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [8]:
generator = pipeline(model=model, tokenizer=tokenizer, task='text-generation')

In [9]:
res = generator(prompt_format.format(prompt=prompt, system_prompt=system_prompt, background=''), do_sample=True, top_p=0.95, max_new_tokens=200)

/Users/dmitriimaksimov/Desktop/GitHub/master_thesis_RAG_LLM/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [10]:
print(res[0]['generated_text'])

<|im_start|>system
Dies ist eine Unterhaltung zwischen einem intelligenten, hilfsbereitem KI-Assistenten und einem Nutzer.
Der Assistent gibt ausführliche, hilfreiche und ehrliche Antworten.<|im_end|>
<|im_start|>user
background: 
Q: Ich habe ein Regal, das 100 kg aushält. Reicht das aus, um die CRIMPING MACHINE EPS 2001 dorthin zu bringen?<|im_end|>
<|im_start|>assistant



In [11]:
res = generator(prompt_format.format(prompt=prompt, system_prompt=system_prompt, background=background), do_sample=True, top_p=0.95, max_new_tokens=200)

KeyboardInterrupt: 

: 

In [ ]:
print(res[0]['generated_text'])

In [4]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

True

In [10]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [23]:
background = """CRIMPING MACHINE EPS 2001
HIGH QUALITY ELECTRICAL CRIMP CONNECTIONS
[1] abhängig von Werkstoff und Kontaktgeometrie
[2] ungefähre Angabe in Basisausstattung, ohne Anbauteile wie z.B. Kontaktstreifenführungsblech und Spulenhalter
TECHNISCHE DATEN
Verarbeit bare Materialien
Leitung Einzeladern oder mehradrige Mantelleitungen
Drahtquerschnitt: < 6 mm² [1]
Kontakte längs- oder quergegurtete Bandware
Leistung und Geometrie 
Antrieb 0,75 kW Drehstrommotor
Motordrehzahl 440 - 2000 U/min
Presskraft 20 kN
Hub 40 mm
Schließhöhe 135,78 mm
Steuerung
Bedienoberfläche STI 2015
Display Touchscreen
Anschlüsse und Schnittstellen
Stromversorgung 230 V, 50/60 Hz
Druckluft 6 bar
Netzwerk RJ-45
Abmessungen und Gewicht
Breite x Tiefe x Höhe 830 x 600 x 700 mm [2]
Gewicht 112 kg [2]
"""

In [24]:
system_prompt = """Dies ist eine Unterhaltung zwischen einem intelligenten, hilfsbereitem KI-Assistenten und einem Nutzer.
Der Assistent gibt ausführliche, hilfreiche und ehrliche Antworten."""
prompt_format = "background: {background}\nQ: {prompt}"
prompt = "Ich habe ein Regal, das 100 kg aushält. Reicht das aus, um die CRIMPING MACHINE EPS 2001 dorthin zu bringen?"

In [27]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt_format.format(prompt=prompt, background='')},
  ]
)

In [29]:
print(response.choices[0].message.content)

Die CRIMPING MACHINE EPS 2001 wiegt etwa 343 kg, daher wäre es nicht ratsam, sie auf ein Regal zu stellen, das nur 100 kg aushält. Es ist wichtig, dass das Regal die angegebene Gewichtskapazität nicht überschreitet, da dies zu strukturellen Schäden und potenziell gefährlichen Situationen führen könnte. Es ist ratsam, ein speziell dafür ausgelegtes Gestell oder Regal zu verwenden, um die CRIMPING MACHINE EPS 2001 sicher zu halten.


In [31]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt_format.format(prompt=prompt, background=background)},
  ]
)

In [32]:
print(response.choices[0].message.content)

Die CRIMPING MACHINE EPS 2001 wiegt 112 kg. Dies überschreitet die maximale Tragfähigkeit Ihres Regals um 12 kg. Es ist wichtig, dass das Regal nicht überladen wird, da dies zu Schäden am Regal oder an der Maschine führen kann. Es wird empfohlen, die Maschine an einem anderen Ort aufzustellen oder ein geeignetes Regal mit einer höheren Tragfähigkeit zu verwenden.
